<div style="text-align: center;">
    <h1> <font style="bold"> Trabajo Práctico 1 </font></h1>
    <h2><font style="bold">Algoritmos de Busqueda en Torres de Hanoi</font></h2>
    <h3><font style="bold">Abril Noguera - Pablo Brahim - Fermin Rodriguez - Kevin Pennington</font></h3>
</div>

En clase presentamos el problema de la torre de Hanoi. Además, vimos diferentes algoritmos de búsqueda que nos 
permitieron resolver este problema. Para este trabajo práctico, deberán implementar un método de búsqueda para 
resolver con 5 discos, del estado inicial y objetivo que se observa en la siguiente imagen:

![Torres Hanoi](./torres.png "Torres Hanoi")


--- 

### 1. ¿Cuáles son los PEAS de este problema? (Performance, Environment, Actuators, Sensors)

#### Supuesto del Problema

El problema de las **Torres de Hanoi** consiste en trasladar un conjunto de **5 discos** desde una **varilla inicial** a una **varilla final**, utilizando una **varilla auxiliar**, siguiendo reglas estrictas:

1. Solo se puede mover un disco a la vez.
2. Cada movimiento translada un disco de una varilla a otra.
2. Un disco no puede colocarse sobre otro más pequeño.
3. El objetivo es mover todos los discos a la última varilla en la menor cantidad de movimientos posible.

Para esta simulación en **Python**, consideramos un **agente computacional** que toma decisiones y ejecuta movimientos basados en un algoritmo de búsqueda, asegurando que los movimientos sean válidos y eficientes. 

El **estado inicial** es un conjunto de 5 discos apilados en la primera varilla, mientras que el **estado final** se alcanza cuando todos los discos están en la tercera varilla, en el mismo orden.

#### Especificación del Entorno (PEAS)

| **Categoría**       | **Descripción** |
|-------------------|-------------------------------|
| **Performance** | - Minimizar la cantidad de movimientos. <br> - Cumplir con las reglas del juego. <br> - Resolver el problema en el menor tiempo computacional posible.  <br> - Resolver el problema en el menor costo computacional (memoria) posible. |
| **Environment** | - Tres varillas representadas como listas. <br> - Cinco discos de diferentes tamaños, representados como enteros segun su tamaño. |
| **Actuators** | - Mover un disco de una varilla a otra, validando reglas. <br> - Ejecutar movimientos en `ActionHanoi`. |
| **Sensors** | - Estado actual de las varillas y posición de los discos. <br> - Validación de movimientos permitidos. <br> - Número de discos que no estan en la varilla objetivo. |


--- 
### 2. ¿Cuáles son las propiedades del entorno de trabajo?

#### Propiedades del entorno

El problema de la Torre de Hanoi se resuelve en un entorno totalmente observable, determinista, secuencial, estático, discreto y con un agente individual.

---
### 3. En el contexto de este problema, establezca cuáles son los: estado, espacio de estados, árbol de búsqueda, nodo de búsqueda, objetivo, acción y frontera.

#### Estado

El estado representa cómo estan distribuidos los discos en los tres postes en un momento determinado, osea, en qué torre esta cada disco y en qué orden.

#### Espacio de estados

El espacio de estados es el conjunto de todas las configuraciones posibles de los discos en todas las torres.

#### Árbol de búsqueda

El árbol de búsqueda es la estructura que representa todas las posibles secuencias de movimientos desde el estado inicial hasta el estado objetivo. En este caso, cada nodo es una configuración de discos, cada rama es una acción que une un estado con otro (haciendo movimientos legales)

#### Nodo de búsqueda

Un nódo de busqueda es un estado específico del árbol donde también sabemos el nodo del que vino, la acción que llevo a ese estado y la profunidad del árbol.

#### Objetivo

El objetivo en este contexto es llegar a un estado en donde todos los discos esten en la torre destino y en el orden correcto (del más grande abajo al más chico arriba de todo)

#### Acción

Una acción es mover el disco superior de una torre a otra, cumpliendo con las reglas del problema.

#### Frontera

La frontera es el conjunto de nodos generados pero aún no explorados en el arbol de búsqueda, osea los siguientes candidatos a ser expandidos (dependiendo el algoritmo utilizado)

--- 

### 4. Implemente algún método de búsqueda. Puedes elegir cualquiera menos búsqueda en anchura primero (el desarrollado en clase). Sos libre de elegir cualquiera de los vistos en clases, o inclusive buscar nuevos.

#### A*

In [ ]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.aima import PriorityQueue

In [ ]:
def heuristic_func(node, goal):

    state = node.state
    count = 0
    for a, b in zip(state.rods[2], goal.rods[2]):
        if a == b:
            count += 1
        else:
            break  
    return len(goal.rods[2]) - count

In [ ]:
def astar_search(heuristic_func, number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    def f(node):
        return node.path_cost + heuristic_func(node, goal_state)
    
    # Creamos el set con estados ya visitados
    explored = set()
    node = NodeHanoi(problem.initial)
    frontier = PriorityQueue(f=f, order='min')

    frontier.append(node)

    node_explored = 0

    # Creamos una cola de prioridad con el nodo inicial
    if problem.goal_test(node.state):
        metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost
            }
        return node, metrics
    
    while len(frontier) != 0:
        node = frontier.pop()[1]
        node_explored += 1
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost
            }
            return node, metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node.expand(problem):
            # Solo si el estado del nodo no fue explorado
            if next_node.state not in explored:
                frontier.append(next_node)
            elif next_node in frontier:
                # Si ya está en la frontera, actualizamos su costo si es menor
                if f(next_node) < frontier[next_node]:
                    del frontier[next_node]
                    frontier.append(next_node)

    # Si no se encontro la solución, devolvemos la métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None
    }
    return None, metrics

In [ ]:
solution, metrics = astar_search(heuristic_func, 5)

In [ ]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 268
states_visited: 169
nodes_in_frontier: 18
max_depth: 31
cost_total: 31.0


In [ ]:
solution.generate_solution_for_simulator()

Execute: 

`poetry shell`

`cd clase2/trabajo_practico_1/simulator`

`python3 simulation_hanoi.py`

--- 
### 5. ¿Qué complejidad en tiempo y memoria tiene el algoritmo elegido? 

A nivel teorico, en el A* hay que buscar referencias.

--- 
### 6. A nivel implementación, ¿qué tiempo y memoria ocupa el algoritmo? (Se recomienda correr 10 veces y calcular promedio y desvío estándar de las métricas).

Para estimar memoria y tiempo
https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

----
### 7. Si la solución óptima es $2^k - 1$ movimientos con *k* igual al número de discos. Qué tan lejos está la solución del algoritmo implementado de esta solución óptima (se recomienda correr al menos 10 veces y usar el promedio de trayecto usado). 